# Code 3-10 to Code 3-20 And Vader Sentiment Code 3-28

In [1]:
import pandas as pd
import stop_words
import re
from nltk.tokenize import sent_tokenize

In [2]:
#stop_words.get_stop_words('en')

In [3]:
t1 = pd.read_csv("Reviews.csv")

In [4]:
t1.shape

(568454, 10)

In [5]:
t1.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
### Get Lexicons

;Citing Lexicons
;
1. Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." 
;       Proceedings of the ACM SIGKDD International Conference on Knowledge  ;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, 
;       Washington, USA, 
2. Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing 
;       and Comparing Opinions on the Web." Proceedings of the 14th 
;       International World Wide Web conference (WWW-2005), May 10-14, 
;       2005, Chiba, Japan.

In [7]:
pos1 = pd.read_csv("positive-words.txt",sep="\t",error_bad_lines=True,encoding='latin1',header=None)
neg1 = pd.read_csv("negative-words.txt",sep="\t",error_bad_lines=True,encoding='latin1',header=None)

pos1.columns = ["words"]
neg1.columns = ["words"]

pos_set = set(list(pos1["words"]))
neg_set = set(list(neg1["words"]))

In [8]:
# In our “t1” dataset we have 2 columns of text - “Text” and “Summary”. 
# We would now combine them and process them into a single column. It is this column that would go through the lexicon mining

In [9]:
t1["full_txt"] = t1["Summary"] + " " + t1["Text"]
t1["full_txt"] = t1["full_txt"].str.lower()
t1["sent_len"] = t1["full_txt"].str.count(" ") + 1


In [10]:
t2 = t1[t1.sent_len>=1]
len(t1),len(t2)

(568454, 568427)

In [11]:
##for meausring accuracy
t2["score_bkt"]="neu"
t2.loc[t2.Score>=4,"score_bkt"] = "pos"
t2.loc[t2.Score<=2,"score_bkt"] = "neg"

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
t3 = t2.sample(frac=0.1)
len(t2),len(t3)

(568427, 56843)

Cleaning text and treating for "nots" in t3 dataset
The idea is to replace bigrams like “did not”, “would not” etc into a single words - “didnt”, “wouldnt”. 

In [14]:
not_list = ["did not","could not","cannot","would not","have not"]


In [15]:
def repl_text(t3,col_to_repl):
    
    t3[col_to_repl] = t3[col_to_repl].str.replace("''","")
    t3[col_to_repl] = t3[col_to_repl].str.replace('[.,]+'," ")
    t3[col_to_repl] = t3[col_to_repl].str.replace("[\s]{2,}"," ")
    

    for i in not_list:
        repl = i.replace("not","").lstrip().rstrip() + "nt"
        repl = repl + " "
        t3[col_to_repl] = t3[col_to_repl].str.replace(i,repl)
    
    return t3

    

In [16]:
t3 = repl_text(t3,"full_txt")
t3 = repl_text(t3,"Summary")

Treating for booster positive and booster negative words
The idea behind is to combine the booster word with subset of positive words (pos_set1) that had a match with that sentence and see the combination together occurs. We have a counter to count both the positive and negative booster hits in this way

In [17]:
booster_words = set(["very","extreme","extremely","huge"])

In [18]:
def booster_chks(com_boost,pos_set1,neg_set1,full_txt_str):
    boost_num_pos = 0
    boost_num_neg = 0
    if(len(pos_set1)>0):
        for a in list(com_boost):
            for b in list(pos_set1):
                wrd_fnd = a + " " + b
                #print (wrd_fnd)
                if(full_txt_str.find(wrd_fnd)>=0):
                    #print (wrd_fnd,full_txt_str,"pos")
                    boost_num_pos = boost_num_pos +1
    if(len(neg_set1)>0):
        for a in list(com_boost):
            for b in list(neg_set1):
                wrd_fnd = a + " " + b
                if(full_txt_str.find(wrd_fnd)>=0):
                    #print (wrd_fnd,full_txt_str,"neg")
                    boost_num_neg = boost_num_neg +1
                    
    return boost_num_pos,boost_num_neg
 

In [19]:
### treating for negation words  - this is different from not words

In [20]:
negation_words = set(["no","dont","didnt","cant","couldnt"])

In [21]:
def neg_chks(com_negation,pos_set1,neg_set1,full_txt_str):
    neg_num_pos = 0
    neg_num_neg = 0
    if(len(pos_set1)>0):
        for a in list(com_negation):
            for b in list(pos_set1):
                wrd_fnd = a + " " + b
                #print (wrd_fnd)
                if(full_txt_str.find(wrd_fnd)>=0):
                    #print (wrd_fnd,full_txt_str,"pos")
                    neg_num_pos = neg_num_pos +1
                    
                    
    if(len(neg_set1)>0):
        for a in list(com_negation):
            for b in list(neg_set1):
                wrd_fnd = a + " " + b
                if(full_txt_str.find(wrd_fnd)>=0):
                    #print (wrd_fnd,full_txt_str,"neg")
                    neg_num_neg = neg_num_neg +1
                    
    return neg_num_pos,neg_num_neg

A small test to demonstrate the function

In [22]:
str_test ="it was given as a gift and the receiver didnt like it i wished i had bought some other kind i was believing that ghirardelli would be the best you could buy but not when it comes to peppermint bark this is not their best effort."
neg_chks({"didnt"},{"like", "best"},{},str_test)

(1, 0)

Treating for excalamations 
Exclamation marks increases the intensity of the underlying sentiment either positive and negative.
We would like to check if there is a exclamation mark in a sentence and if the sentence also has a positive or negative hit

In [23]:
def excl(pos_set1,neg_set1,full_txt_str):
    excl_pos_num=0
    excl_neg_num=0
    
    tok_sent = sent_tokenize(full_txt_str)
    for i in tok_sent:
        if(i.find('!')>=0):
            com_set = set(i.split()) & pos_set1
            if(len(com_set)>0):
                excl_pos_num= excl_pos_num+1
            else:
                com_set1 =set(i.split()) & neg_set1
                if(len(com_set1)>0):
                    excl_neg_num= excl_neg_num+1
    return excl_pos_num,excl_neg_num
            
        


In [24]:
### Logging all the values for future inspection

In [25]:
final_tag_list = []
pos_score_list = []
neg_score_list = []
pos_set_list = []
neg_set_list = []
neg_num_pos_list = []
neg_num_neg_list = []
boost_num_pos_list = []
boost_num_neg_list = []
excl_num_pos_list =[]
excl_num_neg_list =[]

pos_score_list_sum = []
neg_score_list_sum = []
pos_set_list_sum = []
neg_set_list_sum = []
neg_num_pos_list_sum = []
neg_num_neg_list_sum = []
boost_num_pos_list_sum = []
boost_num_neg_list_sum = []
excl_num_pos_list_sum =[]
excl_num_neg_list_sum =[]


In [26]:
### score the reviews

In [27]:
for i,row in t3.iterrows():
    full_txt_str  = row["full_txt"]
    full_txt_set = set(full_txt_str.split())
    sum_txt_str = row["Summary"].lower()
    
    summary_txt_set = set(sum_txt_str.split())
    
    
    sent_len = len(full_txt_set)
    
    ####Postive and Negative sets
    
    pos_set1 = (full_txt_set) & (pos_set)
    neg_set1 = (full_txt_set) & (neg_set)
    
    com_pos_sum = (summary_txt_set) & (pos_set)
    com_neg_sum = (summary_txt_set) & (neg_set)
    
    ####Booster and Negation sets
    
    com_boost = (full_txt_set) & (booster_words)
    com_negation = (full_txt_set) & (negation_words)
    com_boost_sum = (summary_txt_set) & (booster_words)
    com_negation_sum = (summary_txt_set) & (negation_words)
    
    boost_num_pos=0
    boost_num_neg=0
    neg_num_pos=0
    neg_num_neg =0
    excl_pos_num=0
    excl_neg_num = 0
    
    boost_num_pos_sum=0
    boost_num_neg_sum=0
    neg_num_pos_sum=0
    neg_num_neg_sum =0
    excl_pos_num_sum=0
    excl_neg_num_sum = 0
    
    ####Get counters for booster,negation and exclamation sets
    
    if(len(com_boost)>0):
        boost_num_pos,boost_num_neg = booster_chks(com_boost,pos_set1,neg_set1,full_txt_str)
        boost_num_pos_sum,boost_num_neg_sum = booster_chks(com_boost_sum,com_pos_sum,com_neg_sum,sum_txt_str)
        
    if(len(com_negation)>0):
        neg_num_pos,neg_num_neg = neg_chks(com_negation,pos_set1,neg_set1,full_txt_str)
        neg_num_pos_sum,neg_num_neg_sum = neg_chks(com_negation_sum,com_pos_sum,com_neg_sum,sum_txt_str)

        
    if((full_txt_str.find("!")>=0) & ((neg_num_pos+neg_num_neg)==0)):
        excl_pos_num,excl_neg_num = excl(pos_set1,neg_set1,full_txt_str)
        excl_pos_num_sum,excl_neg_num_sum = excl(com_pos_sum,com_neg_sum,sum_txt_str)
        
    ####Compute overall scores
               
    score_pos =  len(pos_set1) + boost_num_pos - 2*neg_num_pos + 2*excl_pos_num   
    score_pos_sum = len(com_pos_sum) + boost_num_pos_sum - 2*neg_num_pos_sum + 2*excl_pos_num_sum
    
    score_pos = score_pos + 1.5*score_pos_sum
            

    score_neg = len(neg_set1) + 3*len(com_neg_sum) + boost_num_neg - 2*neg_num_neg + 2*excl_neg_num
    score_neg_sum = len(com_neg_sum) + boost_num_neg_sum - 2*neg_num_neg_sum + 2*excl_neg_num_sum
    
    score_neg = score_neg + 1.5*score_neg_sum
    
    ####Final Decision
    if((score_pos>0)|(score_neg>0)):
        if((score_neg>score_pos)):
            final_tag = "neg"
        elif(score_pos>score_neg):
            final_tag = "pos"
        else:
            if(full_txt_str.find("!")>=0):
                final_tag = "pos"
            else:
                final_tag = "neu"
                
    else:
        if(full_txt_str.find("!")>=0):
            final_tag="pos"
        else:
            final_tag="neu"
    
    
    ####Log intermitent values for troubleshooting

    final_tag_list.append(final_tag)
    pos_score_list.append(score_pos)
    neg_score_list.append(score_neg)
    
    pos_set_list.append(pos_set1)
    neg_set_list.append(neg_set1)
    neg_num_pos_list.append(neg_num_pos)
    neg_num_neg_list.append(neg_num_neg)
    boost_num_pos_list.append(boost_num_pos)
    boost_num_neg_list.append(boost_num_neg)
    excl_num_pos_list.append(excl_pos_num)
    excl_num_neg_list.append(excl_neg_num)

    pos_set_list_sum.append(com_pos_sum)
    neg_set_list_sum.append(com_neg_sum)
    neg_num_pos_list_sum.append(neg_num_pos_sum)
    neg_num_neg_list_sum.append(neg_num_neg_sum)
    boost_num_pos_list_sum.append(boost_num_pos_sum)
    boost_num_neg_list_sum.append(boost_num_neg_sum)
    excl_num_pos_list_sum.append(excl_pos_num_sum)
    excl_num_neg_list_sum.append(excl_neg_num_sum)
    

In [28]:
t3["final_tags"] = final_tag_list
t3["pos_score"] = pos_score_list
t3["neg_score"] = neg_score_list

t3["pos_set"] = pos_set_list
t3["neg_set"] = neg_set_list

t3["neg_num_pos_count"] = neg_num_pos_list
t3["neg_num_neg_count"] = neg_num_neg_list

t3["boost_num_pos_count"] = boost_num_pos_list
t3["boost_num_neg_count"] = boost_num_neg_list


t3["pos_set_sum"] = pos_set_list_sum
t3["neg_set_sum"] = neg_set_list_sum

t3["neg_num_pos_count_sum"] = neg_num_pos_list_sum
t3["neg_num_neg_count_sum"] = neg_num_neg_list_sum

t3["boost_num_pos_count_sum"] = boost_num_pos_list_sum
t3["boost_num_neg_count_sum"] = boost_num_neg_list_sum

t3["excl_num_pos_count"] = excl_num_pos_list
t3["excl_num_neg_count"] = excl_num_neg_list

t3["excl_num_pos_count_sum"] = excl_num_pos_list_sum
t3["excl_num_neg_count_sum"] = excl_num_neg_list_sum


In [29]:
### Measuring model performance

In [30]:
from sklearn.metrics import accuracy_score
print (accuracy_score(t3["score_bkt"],t3["final_tags"]))
#0.8003448093872596

from sklearn.metrics import f1_score
f1_score(t3["score_bkt"],t3["final_tags"], average='macro')
#0.502500231042986

rows_name = t3["score_bkt"].unique()

from sklearn.metrics import confusion_matrix
cmat = pd.DataFrame(confusion_matrix(t3["score_bkt"],t3["final_tags"], labels=rows_name, sample_weight=None))
cmat.columns = rows_name 
cmat["act"] = rows_name
cmat


0.797987439086607


,neg,pos,neu,act
0,4225,3491,504,neg
1,2669,40884,738,pos
2,1047,3034,251,neu


In [31]:
t4 = t3.loc[(t3.score_bkt!="neu") & (t3.final_tags!="neu")].reset_index()
print (accuracy_score(t4["score_bkt"],t4["final_tags"]))
print (f1_score(t4["score_bkt"],t4["final_tags"],average='macro'))

0.879849421677817
0.7541568295782543


In [32]:
pd.crosstab(t4["score_bkt"],t4["final_tags"])

final_tags,neg,pos
score_bkt,,
neg,4225,3491
pos,2669,40884


In [33]:
req_cols_subset = ['Summary','full_txt','final_tags','score_bkt','pos_score','neg_score','pos_set','neg_set','Score']

In [34]:
pd.options.display.max_colwidth=1000
t3.loc[(t3.score_bkt!=t3.final_tags) & (t3.score_bkt=="neu"),req_cols_subset]

,Summary,full_txt,final_tags,score_bkt,pos_score,neg_score,pos_set,neg_set,Score
226818,Chocolate-y Snack Food,"chocolate-y snack food overview: a decent low calorie chocolate-y snack<br /><br />survey participants and methodology: krave double chocolate cereal was tested on six adults between the ages of 32 to 65 a seventh adult aged 22 pledged allegiance to cocoa puffs and refused to participate only one participant (me) was willing to try krave with skim milk at breakfast the other five sampled the cereal dry straight out of the box or took handfuls from a communal serving bowl a canine participant ate whatever fell to the floor <br /><br />results: opinions were split according to sex and to a lesser degree age demographics females were more positive about the chocolate flavor and the airy but crunchy texture males were fairly united in their overall opinion of ""meh"" but couldnt otherwise precisely express themselves one male continued to eat large handfuls of krave even after stating a preference for cocoa puffs which was remembered as being sweeter and had more of a milk chocolate fla...",pos,neu,8.0,4.0,"{precisely, enthusiastic, positive, sweet, fairly, willing, robust, decent}","{fell, split, refused, fat}",3
491088,Kinda dissapointing,kinda dissapointing i love gummy candy and find it hard to resist the classic haribo gummy bears and i regret straying from them to try these the flavors aren't very distinct and area all kinda icky the strawberry and orange are very bitter the lemon tastes like pynsol smells the only one i cant really complain about is grape,neg,neu,3.0,7.0,"{love, classic, like}","{lemon, smells, bitter, hard, complain, regret}",3
290957,It's marketed water that is not at an ideal pH for the body,it's marketed water that is not at an ideal ph for the body this water purportedly contains 9 5 ph however the purported benefits of high ph water are not demonstrated in scientific literature only in this company's marketing materials the fact is that the ideal ph level of drinking water should be between 6-8 5 the body maintains a ph equilibrium on a constant basis that is not affected by water consumption of higher or lower ph unless it's consistent consumption on a regular basis in fact our stomachs have a naturally low ph level of about 2 which is helpful for food digestion and our stomachs will self-adjust to maintain this naturally low ph level <br /><br />yes it's water and amazon will ship 12 bottles to you for about 2 bucks a bottle if you do choose to get this monster shipment also consider purchasing carbon off-sets,pos,neu,5.5,1.0,"{helpful, consistent, ideal, benefits}",{monster},3
235444,Not my favorite tasting jimmies,not my favorite tasting jimmies pros - good price for its size fast delivery <br />con - not the best tasting jimmies still better than getting the small grocery store size,pos,neu,7.5,0.0,"{pros, fast, good, best, better, favorite}",{},3
266632,Good Coffee but Order was partially wrong,good coffee but order was partially wrong i ordered the 2-pack of sumatra coffee beans but received one sumatra and one dark french roast probably a four or five star coffee if order had been correct,neg,neu,3.5,6.5,"{good, correct}","{wrong, dark}",3
412775,Not as good as other flavors,not as good as other flavors but it could just be my taste buds (or lack there of) i like the stonewall's jerquee items very much just not this particulare kind,pos,neu,3.5,1.0,"{good, like}",{lack},3
269240,Look good to me but expensive,"look good to me but expensive best feline friend cat food cost more for less food i didnt try it but my cat eats few bites and look at me for something else i found that it has a very strong smell of fish compared to other brand in other word may be the food is too concentrated despite the much higher cost i would have given my cat this brand if he would have liked it sorry ""best feline"" but i didnt decide if you want more details ask wicket my cat",neg,neu,6.5,7.5,"{good, str

In [30]:
t3.to_csv("lexicon_sent_processed.csv")

In [ ]:
##implementing vader

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
score = analyser.polarity_scores("I am good")

In [36]:
score

{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound': 0.4404}